# Восстановление 3D-трека из проекций с регуляризацией (cvxpy)

In [ ]:
import numpy as np
import cvxpy as cp
from scipy.sparse import csr_matrix
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from math import atan2, acos
from sklearn.linear_model import LinearRegression

In [ ]:
def get_volume_for_event(calo_data, event_id=0, z_size=44, y_size=96, x_size=96):
    mask = calo_data['event_ID'] == event_id
    x = calo_data['index_along_x'][mask].astype(int)
    y = calo_data['index_along_y'][mask].astype(int)
    z = calo_data['layer'][mask].astype(int)
    e = calo_data['energy_release'][mask]
    volume = np.zeros((z_size, y_size, x_size), dtype=np.float32)
    for xi, yi, zi, ei in zip(x, y, z, e):
        if 0 <= xi < x_size and 0 <= yi < y_size and 0 <= zi < z_size:
            volume[zi, yi, xi] += ei
    return volume

In [ ]:
def compute_projections(volume):
    proj_x = np.sum(volume, axis=2).reshape(-1)
    proj_y = np.sum(volume, axis=1).reshape(-1)
    return np.concatenate([proj_x, proj_y])

In [ ]:
def build_projection_matrix(Z, Y, X):
    from scipy.sparse import lil_matrix
    A = lil_matrix((Z * (Y + X), Z * Y * X), dtype=np.float32)
    row = 0
    for z in range(Z):
        for y in range(Y):
            for x in range(X):
                col = z * Y * X + y * X + x
                A[row, col] = 1.0
            row += 1
        for x in range(X):
            for y in range(Y):
                col = z * Y * X + y * X + x
                A[row, col] = 1.0
            row += 1
    return A.tocsr()

In [ ]:
def smoothness_matrix(Z, Y, X):
    rows = []
    for z in range(Z - 1):
        for y in range(Y):
            for x_ in range(X):
                idx1 = z * Y * X + y * X + x_
                idx2 = (z + 1) * Y * X + y * X + x_
                row = np.zeros(Z * Y * X)
                row[idx1] = -1
                row[idx2] = +1
                rows.append(row)
    return csr_matrix(np.array(rows))

In [ ]:
def compute_track_angles(volume):
    Z, Y, X = volume.shape
    z_vals, x_centers, y_centers = [], [], []
    for z in range(Z):
        layer = volume[z]
        total = np.sum(layer)
        if total > 1e-6:
            y_coords, x_coords = np.indices(layer.shape)
            x_center = np.sum(x_coords * layer) / total
            y_center = np.sum(y_coords * layer) / total
            z_vals.append(z)
            x_centers.append(x_center)
            y_centers.append(y_center)
    z_vals = np.array(z_vals).reshape(-1, 1)
    reg_x = LinearRegression().fit(z_vals, x_centers)
    reg_y = LinearRegression().fit(z_vals, y_centers)
    dx, dy, dz = reg_x.coef_[0], reg_y.coef_[0], 1.0
    norm = np.sqrt(dx**2 + dy**2 + dz**2)
    return acos(dz / norm), atan2(dy, dx)

In [ ]:

# === Шаг 4: Обработка батча событий ===
def get_true_angles(calo_data, event_id):
    theta = calo_data["theta_0"][event_id]
    phi = calo_data["phi_0"][event_id]
    vx = np.sin(theta) * np.cos(phi)
    vy = np.sin(theta) * np.sin(phi)
    vz = np.cos(theta)
    norm = np.sqrt(vx**2 + vy**2 + vz**2)
    return np.arccos(vz / norm), np.arctan2(vy, vx)

def filter_projection_system(A, b, threshold=0.1):
    keep_mask = np.abs(b) > threshold
    return A[keep_mask], b[keep_mask]

Z, Y, X = 44, 96, 96
num_events = 10
lambda_reg = 0.1
theta_errors = []
phi_errors = []

calo_data = np.load("calorimeter_response.npy", allow_pickle=True).item()

for event_id in range(num_events):
    volume = get_volume_for_event(calo_data, event_id, Z, Y, X)
    b = compute_projections(volume)
    A = build_projection_matrix(Z, Y, X)
    A_clean, b_clean = filter_projection_system(A, b)
    A_sparse = csr_matrix(A_clean)

    x = cp.Variable(A_sparse.shape[1], nonneg=True)
    D = smoothness_matrix(Z, Y, X)
    objective = cp.Minimize(cp.sum_squares(A_sparse @ x - b_clean) + lambda_reg * cp.sum_squares(D @ x))
    problem = cp.Problem(objective)
    problem.solve(solver=cp.OSQP, verbose=False)

    volume_hat = x.value.reshape((Z, Y, X))
    theta_hat, phi_hat = compute_track_angles(volume_hat)
    theta_true, phi_true = get_true_angles(calo_data, event_id)

    theta_errors.append(abs(theta_hat - theta_true))
    phi_errors.append(abs(phi_hat - phi_true))

# Визуализация ошибок
import matplotlib.pyplot as plt
plt.plot(theta_errors, label="theta error")
plt.plot(phi_errors, label="phi error")
plt.xlabel("Event ID")
plt.ylabel("Angle error (radians)")
plt.legend()
plt.grid()
plt.title("Ошибка восстановления углов по батчу событий")
plt.show()
